## Import Statements

In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "16"  # Set 16 threads for OpenBLAS
os.environ["OPENBLAS_NUM_THREADS"] = "16"  # Set 16 threads for OpenBLAS

from findiff.operators import FinDiff, Identity, Coef
from findiff.pde import *
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import pickle
import pandas as pd
import cupy as cp
from cupyx.scipy.sparse import csr_matrix as csr_gpu_matrix
from cupyx.scipy.sparse import coo_matrix as coo_matrix_real
from scipy.sparse import csr_matrix as csr_matrix_cpu
from scipy.sparse.linalg import spsolve
from cupyx.scipy.sparse import diags
from cupyx.scipy.sparse.linalg import LinearOperator, cg, spilu, gmres
import time as timer

c:\Users\jslon\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Hyperparams

In [2]:
#Space Grid
rmin = 1e-6
rmax = 1.1
nr = 50
r, dr = np.linspace(rmin, rmax, nr, retstep = True)

#Time Grid
tmin = 2.9
tmax = 4.0
nt = 1000
time, dt = np.linspace(tmin, tmax, nt, retstep = True)

#Number of Samples
iterations = 10000

#Files
Input_File = 'ProfileDataset622v2.pickle'
Output_File = 'SolvedDataset622v2.pickle'

#Flags
save = 1

## Matrix Creation Functions

In [3]:
def M1_Create(nr=50, nt=1000, dt=1):
    '''Docstring WIP'''
    #Define Matrix Size
    N = nr*nt
    
    #Define Scaling Constant
    a = 1/(2*dt)
    
    #Second Matrix
    err_rows1 = cp.concatenate([cp.arange(0,nr,1), cp.repeat(cp.arange(0,N,1),2), cp.arange(N-nr,N,1)])
    err_cols1 = cp.concatenate([cp.arange(0,nr,1), cp.tile(cp.array([nr,2*nr]),nr) + cp.repeat(cp.arange(0,nr,1),2), cp.tile(cp.array([0,nr*2]),N-2*nr) + cp.repeat(cp.arange(0,N-2*nr,1),2), cp.tile(cp.array([N-3*nr,N-2*nr]),nr) + cp.repeat(cp.arange(0,nr,1),2), cp.arange(N-nr,N,1)])
    err_vals1 = a*cp.concatenate([-3*cp.ones(nr), cp.tile(cp.array([4,-1]), nr), cp.tile(cp.array([-1,1]), N-2*nr), cp.tile(cp.array([1,-4]), nr), 3*cp.ones(nr)])
    coo_extra1 = coo_matrix_real((err_vals1, (err_rows1, err_cols1)), shape=(N, N))
    
    return coo_extra1.tocsr()

In [4]:
def M2_Create(nr=50, nt=1000, dr=1, C=1):
    '''Docstring WIP'''
    #Define Matrix Size
    N = nr*nt
    
    #Define Scaling Constant
    a0 = 1/(dr**2)
    a1 = a0*cp.array(cp.array(C.ravel()))
    
    #Create diagonal values directly on the GPU
    main_diag = a1*cp.concatenate([cp.array([2]), -2*cp.ones(N - 2), cp.array([2])])
    upper_diag = a1[:-1]*cp.concatenate([cp.array([-5]), cp.ones(N - 2)])
    lower_diag = a1[1:]*cp.concatenate([cp.ones(N - 2), cp.array([-5])])
    
    #Formatting
    diagonals = [main_diag, upper_diag, lower_diag]
    offsets = [0, 1, -1]
    
    # Create a diagonal sparse matrix
    csr_gpu1 = diags(diagonals, offsets, format='csr')
    
    #Second Matrix
    err_rows1 = cp.sort(cp.concatenate([cp.tile(cp.concatenate([cp.arange(nr,N,nr), cp.arange(nr-1,N-1,nr)]),4),cp.array([0,0,N-1,N-1])])) #Optimize cp.repeat()?
    err_cols1 = cp.concatenate([cp.array([2,3]), cp.repeat(cp.arange(nr-4,N-nr-3,nr),8) + cp.tile(cp.array([0,1,2,3,4,5,6,7]),nt-1), cp.array([N-4,N-3])]) #Optimize, No comprehension
    err_vals1 = a1[err_rows1]*cp.concatenate([cp.array([4,-1]), cp.tile(cp.array([-1,4,-6,4,4,-6,4,-1]),nt-1), cp.array([-1,4])])
    coo_extra1 = coo_matrix_real((err_vals1, (err_rows1, err_cols1)), shape=(N, N))
    
    #Third Matrix
    err_rows2 = cp.sort(cp.concatenate([cp.arange(nr-1,N-1,nr), cp.arange(nr,N,nr)]))
    err_cols2 = cp.vstack((cp.arange(nr,N-nr+1,nr), cp.arange(nr-1,N-nr,nr))).T.ravel()
    err_vals2 = -a1[err_rows2]*cp.ones(2*nt-2)
    coo_extra2 = coo_matrix_real((err_vals2, (err_rows2, err_cols2)), shape=(N, N))
    
    csr_gpu = csr_gpu1 + coo_extra1.tocsr() + coo_extra2.tocsr()
    
    return csr_gpu

In [5]:
def M3_Create(nr=50, nt=10000, dr=1, C=1):
    '''Docstring WIP'''
    #Define Matrix Size
    N = nr*nt
    
    #Define Scaling Constant
    a0 = 1/(2*dr)
    a1 = a0*cp.array(C.ravel())
    
    #Create diagonal values directly on the GPU
    main_diag = a1*cp.concatenate([cp.array([-3]), cp.zeros(N-2), cp.array([3])])
    upper_diag = a1[:-1]*cp.concatenate([cp.array([4]), cp.ones(N-2)])
    lower_diag = a1[1:]*cp.concatenate([-cp.ones(N-2), cp.array([-4])])
    
    #Formatting
    diagonals = [main_diag, upper_diag, lower_diag]
    offsets = [0, 1, -1]
    
    # Create a diagonal sparse matrix
    csr_gpu1 = diags(diagonals, offsets, format='csr')
    
    #Second Matrix
    rows = cp.concatenate([cp.array([0,N-1]), cp.repeat(cp.arange(nr-1,N-nr,nr),4), cp.repeat(cp.arange(nr,N-nr+1,nr),4)])
    cols = cp.concatenate([cp.array([2,N-3]), cp.repeat(cp.arange(nr-1,N-nr,nr),4) + cp.tile(cp.array([-2,-1,0,1]),nt-1), cp.repeat(cp.arange(nr,N-nr+1,nr),4) + cp.tile(cp.array([0,1,2,-1]),nt-1)])
    values = a1[rows]*cp.concatenate([cp.array([-1,1]), cp.tile(cp.array([1,-3,3,-1]),nt-1), cp.tile(cp.array([-3,3,-1,1]),nt-1)])
    coo_extra = coo_matrix_real((values, (rows, cols)), shape=(N, N))
    
    csr_gpu = csr_gpu1 + coo_extra.tocsr()
    
    return csr_gpu

In [6]:
def M4_Create(nr=50, nt=10000, C=1):
    '''Docstring WIP'''
    #Define Matrix Size
    N = nr*nt
    
    #Define Scaling Constant
    a1 = cp.array(C.ravel())
    
    #Create diagonal values directly on the GPU
    main_diag = a1*cp.ones(N)
    
    # Create a diagonal sparse matrix
    csr_gpu = diags(main_diag, 0, format='csr')
    
    return csr_gpu

In [7]:
def MBCL_Create(nr=50, nt=1000, dr=1):
    '''Docstring WIP'''
    #Define Matrix Size
    N = nr*nt
    a = 1/(2*dr)
    Rows1 = cp.concatenate([cp.arange(1,nr,1), cp.arange(2*nr-1,N,nr), cp.repeat(cp.arange(0,N,nr),3)])
    Cols1 = cp.concatenate([cp.arange(1,nr,1), cp.arange(2*nr-1,N,nr), cp.repeat(cp.arange(0,N,nr),3) + cp.tile(cp.array([0,1,2]),nt)])
    Vals1 = cp.concatenate([cp.ones(nt+nr-2), a*cp.tile(cp.array([-3,4,-1]),nt)])
    M1 = coo_matrix_real((Vals1, (Rows1, Cols1)), shape=(N, N)).tocsr()
    return M1

In [8]:
def MBCR_Create(nr=50, nt=1000, CMBCR=1):
    #Define Matrix Size
    N = nr*nt
    Rows2 = cp.concatenate([cp.arange(1,nr,1), cp.arange(2*nr-1,N,nr)])
    Cols2 = cp.zeros(nt+nr-2)
    Vals2 = cp.concatenate([cp.array(CMBCR[1:]),CMBCR[-1]*cp.ones(nt-1)])
    M2 = coo_matrix_real((Vals2, (Rows2, Cols2)), shape=(N, 1)).tocsr()
    return M2

In [9]:
def ReadPickle(filename: str) -> dict:
  '''Reads in data from given pickle files, outputs a dictionary'''
  try:
    Data = pd.read_pickle(filename)
  except FileNotFoundError:
    raise FileNotFoundError(f'Error reading {filename}')
  return Data
def convert_seconds(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = seconds % 60
    return hours, minutes, secs

In [10]:
class PDE_v2(object):
    def __init__(self, lhs, rhs, bcsL, bcsR):
        self.lhs = lhs
        #self.lhs.matrix = profile(self.lhs.matrix)
        #self.lhs.left.matrix = profile(self.lhs.left.matrix)
        self.rhs = rhs
        self.bcsL = bcsL
        self.bcsR = bcsR
        self._L = None
        
    def solve_v2(self):
        shape = self.rhs.shape
        if self._L is None:
            self._L = self.lhs # expensive operation
            
        L_GPU = self._L
        bcs_lhs_gpu = self.bcsL


        f = cp.array(self.rhs.reshape(-1, 1))

        nz = self.bcsL.tocoo().row

        L_GPU[nz, :] = bcs_lhs_gpu[nz, :]

        f[nz] = cp.array(self.bcsR[nz].toarray()).reshape(-1, 1)

        L = L_GPU.get()
        u = spsolve(L, cp.asnumpy(f)).reshape(shape)
        return u

In [11]:
acc = 2 # global FinDiff numerical accuracy order

#Differential Operators
d_dr = FinDiff(1, r, 1, acc = acc)
d2_dr2 = FinDiff(1, r, 2, acc = acc)

shape = (len(time), len(r)) # (1000,50)

In [12]:
def FinalBuilder(nr, nt, r, dr, time, dt, SRin, STin, Din, Vin, N0in):
    '''Docstring WIP'''
    SGrid = np.outer(STin, SRin)
    DGrid = np.outer(np.ones(len(time)), Din)
    VGrid = np.outer(np.ones(len(time)), Vin)

    gradD = d_dr(DGrid)
    gradv = d_dr(VGrid)

    BLinear = (gradD - VGrid)
    CM3 = BLinear + DGrid/r
    CM4 = gradv + VGrid/r

    M1m = M1_Create(nr=nr, nt=nt, dt=dt)
    M2m = M2_Create(nr=nr, nt=nt, dr=dr, C=DGrid)
    M3m = M3_Create(nr=nr, nt=nt, dr=dr, C=CM3)
    M4m = M4_Create(nr=nr, nt=nt, C=CM4)
    MFm = M1m - M2m - M3m + M4m

    BCL = MBCL_Create(nr=nr, nt=nt, dr=dr)
    BCR = MBCR_Create(nr=nr, nt=nt, CMBCR=N0in)

    pde = PDE_v2(MFm.copy(), SGrid.copy(), BCL.copy(), BCR.copy())
    u = pde.solve_v2()
    return u

In [13]:
data_in = ReadPickle(Input_File)
data_out = []

print('Starting Solver')
t1 = timer.time()
for i in range(iterations):
    Dict_in = data_in[i]
    SRin, STin, Din, Vin, N0in = Dict_in['SR'], Dict_in['ST'], Dict_in['D'], Dict_in['V'], Dict_in['N0']
    N = FinalBuilder(nr, nt, r, dr, time, dt, SRin, STin, Din, Vin, N0in)
    Dict_Out = {'N':N,'D':Din,'V':Vin, 'SR':SRin, 'ST':STin,'N0':N0in}
    data_out.append(Dict_Out)
if save:
    with open(Output_File, 'wb+') as f:
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(data_out, f, pickle.HIGHEST_PROTOCOL)
t2 = timer.time()
print('Solver Finished')

delta_t = t2 - t1
hrs, mins, secs = convert_seconds(delta_t)
print(f"Total Time Taken: {hrs}hr {mins}min {np.round(secs,3)}s")
print(f"Time Per Iteration: {np.round((delta_t)/iterations,5)}s")


Starting Solver
Solver Finished
Total Time Taken: 1hr 11min 23.124s
Time Per Iteration: 0.42831s


## Add Boundary Checks

In [14]:
Diagnose = 0
if Diagnose:
        SGrid = np.outer(np.ones(len(time)), Sin)
        DGrid = np.outer(np.ones(len(time)), Din)
        VGrid = np.outer(np.ones(len(time)), Vin)

        gradD = d_dr(DGrid)
        gradv = d_dr(VGrid)

        BLinear = (gradD - VGrid)
        CM3 = BLinear + DGrid/r
        CM4 = gradv + VGrid/r

        M1m = M1_Create(nr=nr, nt=nt, dt=dt)
        M2m = M2_Create(nr=nr, nt=nt, dr=dr, C=DGrid)
        M3m = M3_Create(nr=nr, nt=nt, dr=dr, C=CM3)
        M4m = M4_Create(nr=nr, nt=nt, C=CM4)
        MFm = M1m - M2m - M3m + M4m
        
        L = ( FinDiff(0, time, 1, acc = acc) 
                - Coef(DGrid)*FinDiff(1, r, 2, acc = acc)
                - Coef(BLinear + DGrid/r)*FinDiff(1, r, 1, acc = acc)
                + Coef(gradv + VGrid/r)*Identity() ) 
        MFt = L.matrix(shape)
        M1t = FinDiff(0, time, 1, acc = acc).matrix(shape)
        M2t = (Coef(DGrid)*FinDiff(1, r, 2, acc = acc)).matrix(shape)
        M3t = (Coef(BLinear + DGrid/r)*FinDiff(1, r, 1, acc = acc)).matrix(shape)
        M4t = (Coef(gradv + VGrid/r)*Identity()).matrix(shape)
        
        MDf = np.abs(MFm - csr_gpu_matrix(MFt))
        MD1 = np.abs(M1m - csr_gpu_matrix(M1t))
        MD2 = np.abs(M2m - csr_gpu_matrix(M2t))
        MD3 = np.abs(M3m - csr_gpu_matrix(M3t))
        MD4 = np.abs(M4m - csr_gpu_matrix(M4t))
        
        print(f"Total Real: {np.abs(MFt).sum()}")
        print(f"Total Calc: {np.abs(MFm).sum()}")
        print('')
        print(f"Total Diff: {MDf.sum()}")
        print(f"Diff Term1: {MD1.sum()}")
        print(f"Diff Term2: {MD2.sum()}")
        print(f"Diff Term3: {MD3.sum()}")
        print(f"Diff Term4: {MD4.sum()}")
        print(f"Diff Ratio: {MDf.sum()/np.abs(MFt).sum()}")

In [15]:
Check = 0
total_diff = 0
total = 0
if Check:
    data_vals = ReadPickle('Solved_Dataset_3.pickle')
    for i in range(iterations):
        N = data_vals[i]['N']
        total += np.abs(N).sum()
        diff = np.abs(N - data_out[i]).sum()
        total_diff += diff
    print(f"Average Diff: {total_diff/iterations}")
    print(f"Ratio Diff: {total_diff/total}")
